In [ ]:
import argparse
import sys
import os
from skimage import io

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils
import matplotlib.patches as mpatches


# from scheduler import CycleScheduler
from pt_utils import  Embeddings, Trainer, VQVAE, data_sampler, Vqvae2Adaptive
from torch.utils import data
from torch import distributed as dist

from umap import UMAP
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from collections import Counter
from skimage import transform, metrics
import skimage
import pandas as pd
import numpy as np
import glob

from torchsummary import summary

from tqdm.notebook import trange, tqdm
import torchvision
from PIL import Image

import numpy as np
from sklearn.decomposition import PCA
from scipy.ndimage.filters import gaussian_filter
import scipy as sp
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import joblib
import pickle

seed = 51
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# dataset_path = '../datasets/bc_right_sub_left_minmax_4x_360'
# dataset_path = '../datasets/bc_left_sub_right_0.5_4x_360'
# dataset_path = '../datasets/bc_right_sub_left_0.5_4x_360'
# dataset_path = '../datasets/bc_left_sub_right_minmax_4x_360'
# dataset_path = '../datasets/bc_left_4x_360'
dataset_path = '../datasets/original/o_bc_left_9x_512_360'

resize_shape = (512, 512)

transform = transforms.Compose(
    [   
        # transforms.CenterCrop((256,256)),
        # transforms.functional.invert,
        transforms.ToTensor(),
        # transforms.Grayscale(),
        transforms.Normalize(0.5, 0.5),
    ]
)

dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [ ]:
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_4x_360/vqvae_004_train_0.02043_test_0.02022.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_sub_right_minmax_4x_360/vqvae_002_train_0.04045_test_0.03895.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_right_sub_left_minmax_4x_360/vqvae_003_train_0.04287_test_0.04129.pt'
# model_file = '../data/logs/vq-vae-2/4x_emb_dim_32/weights/vqvae_011_train_0.00375_test_0.00375.pt'
# model_file = '../data/logs/vq-vae-2/4x/weights/vqvae_013_train_0.0038_test_0.0038.pt'
model_file = 'data/logs/emb_dim_8_n_embed_8192_bc_left_9x_512_360/vqvae_002_train_0.02079_test_0.02053.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_9x_512_360/vqvae_002_train_0.01976_test_0.01984.pt'
# model_file = 'data/logs/emb_dim_2_n_embed_8192_bc_left_9x_512_360/vqvae_002_train_0.01791_test_0.01756.pt'

# model_file = 'data/logs/emb_dim_1_n_embed_8192_o_bc_left_9x_256_360_1/vqvae_003_train_0.01975_test_0.01967.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_o_bc_left_9x_512_360_1/vqvae_003_train_0.01967_test_0.01958.pt'



device = "cuda"

# model =    VQVAE(in_channel=1,
model =    Vqvae2Adaptive(in_channel=3,
                   channel=128,
                   n_res_block=6,
                   n_res_channel=32,
                   embed_dim=8,
                   n_embed=8192,
                   decay=0.99).to(device)

# model =    Vqvae2Adaptive(in_channel=3,
#                  channel=128,
#                  n_res_block=6,
#                  n_res_channel=32,
#                  embed_dim=1,
#                  n_embed=8192,
#                  decay=0.99).to(device)

model.load_state_dict(torch.load(model_file, map_location=torch.device('cuda')))

In [ ]:
dataset_path = dataset.__dict__['root']
classes_folders = os.listdir(dataset_path)
classes_folders_images = [os.listdir(dataset_path + '/' + folder) for folder in classes_folders]
classes_folders_images_num = [len(os.listdir(dataset_path + '/' + folder)) for folder in classes_folders]
img_transform = dataset.__dict__['transform']

i=0
j=0

image_path = dataset_path + '/' + classes_folders[i] + '/' + classes_folders_images[i][j]

img = Image.open(image_path)
image = transform(img)
image = image.unsqueeze(0).to(device)

# print(image.shape)

quant_t, quant_b, diff, _, indx_b = model.encode(image)

print(quant_t.shape)
# print(quant_t.cpu().detach().numpy().flatten().shape)
print(quant_b.shape)
# print(quant_b.cpu().detach().numpy().flatten().shape)

In [ ]:
res=model.decode(quant_t, quant_b)

res.shape

In [ ]:
plt.imshow(res.cpu().detach().numpy()[0][0],cmap='gray')

In [ ]:
plt.imshow(img,cmap='gray')

In [ ]:
embs_t, embs_b = Embeddings.get_vqvae2_embs(model, dataset)


In [ ]:
np.save(f'data/embs/embs_t_9x_256_360_1_emb_dim_1_n_embed_8192_left_model_left.npy', embs_t)
np.save('data/embs/embs_b_9x_256_360_1_emb_dim_1_n_embed_8192_left_model_left.npy', embs_b)

In [ ]:
dataset_path = dataset.__dict__['root']
classes_folders = os.listdir(dataset_path)
classes_folders_images = [os.listdir(dataset_path + '/' + folder) for folder in classes_folders]
classes_folders_images_num = [len(os.listdir(dataset_path + '/' + folder)) for folder in classes_folders]
img_transform = dataset.__dict__['transform']

i=0
j=0

image_path = dataset_path + '/' + classes_folders[i] + '/' + classes_folders_images[i][j]

image = Image.open(image_path).convert("RGB")
image = img_transform(image).unsqueeze(0).to(device)

quant_t, quant_b, diff, id_t, id_b = model.encode(image,n_embedded_l=-2, dim_l=-2)

In [ ]:
dim_l=-6
# dim_l=None
n_embedded_l=-2048
# n_embedded_l=None

embs_t, embs_b = Embeddings.get_vqvae2_embs_adaptive(model, dataset, n_embedded_l=n_embedded_l,dim_l=dim_l)


# np.save(f'data/embs/embs_t_9x_512_360_emb_dim_8{dim_l}_n_embed_8192{n_embedded_l}_left_model_left.npy', embs_t)
# np.save('data/embs/embs_b_9x_512_360_emb_dim_8-7_n_embed_8192_left_model_left.npy', embs_b)

print(embs_t.shape)

In [ ]:
embs_b.shape

## Features visualization

In [ ]:
embs_t=np.load('data/embs/embs_t_9x_512_360_1_emb_dim_1_n_embed_8192_left_model_left.npy')
embs_b=np.load('data/embs/embs_b_9x_512_360_1_emb_dim_1_n_embed_8192_left_model_left.npy')

In [ ]:
seed = 52
np.random.seed(seed)
torch.manual_seed(seed)

umap_2d = UMAP(random_state=seed,n_components=2)

# embs_b=np.array(embs_b)
# embs_t=np.array(embs_t)

# umaped_vct_2d_t = umap_2d.fit_transform(embs_t)
# umaped_vct_2d_b = umap_2d.fit_transform(embs_b)

umaped_vct_tb=umap_2d.fit_transform(np.concatenate([embs_t,embs_b],axis=1))

In [ ]:
# reducer = umap.UMAP()

# embedding = reducer.fit_transform(X, y)

f_name = 'umap_tb_9x_256_360_1_emb_dim_1_n_embed_8192_left_model_left.sav'
pickle.dump(umap_2d, open(f_name, 'wb'))

In [ ]:
f_name = 'umap_tb_9x_512_360_1_emb_dim_1_n_embed_8192_left_model_left.sav'
umaped_vct_tb_loaded = pickle.load((open(f_name, 'rb')))

In [ ]:
umaped_vct_tb=umaped_vct_tb_loaded.transform(np.concatenate([embs_t,embs_b],axis=1))

In [ ]:
test_pts=np.array([[0,4.5]])

quant_tb=umaped_vct_tb_loaded.inverse_transform(test_pts)

res=model.decode_li(torch.from_numpy(quant_tb).to(device), N=4096, t_size=64, b_size=128)

plt.imshow(np.swapaxes(res.cpu().detach().numpy()[0],0,-1),cmap='gray')
plt.savefig('0_4.5_reconstruct_512.png',bbox_inches='tight')
plt.show()

In [ ]:
seed=51


types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

classes = dataset.__dict__['classes']

legend = [cls + '\n' + types_dict[cls] for cls in classes]

# name_1 = f'embs_space_seed={seed}_t_512_4x_360_minmax_emb_dim_1_n_embed_8192_right_sub_left_unique_model'
# name_1 = f'embs_space_seed={seed}_t_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left' 
name_1 = f'embs_space_seed={seed}_t_256_9x_360_emb_dim_1_n_embed_8192_left_model_left' 

# name_2 = f'embs_space_seed={seed}_b_512_4x_360_minmax_emb_dim_1_n_embed_8192_right_sub_left_unique_model'
# name_2 = f'embs_space_seed={seed}_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left'
name_2 = f'embs_space_seed={seed}_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left'


r_shape = (len(classes), -1, 2)

# save=True
save=False

# Embeddings.plot_2d_scatter_embs(embs_scatter=umaped_vct_2d_t, legend=legend, r_shape=r_shape, save=save, name=name_1, N=10, M=10)
Embeddings.plot_2d_scatter_embs(embs_scatter=umaped_vct_tb, legend=legend, r_shape=r_shape, save=save, name=name_1, N=10, M=10)
# Embeddings.plot_2d_scatter_embs(embs_scatter=umaped_vct_2d_b, legend=legend, r_shape=r_shape, save=save, name=name_2, N=10, M=10)

In [ ]:
embs_t.shape

In [ ]:
embs_b.shape

In [ ]:
umaped_vct_tb.reshape((5,3240,2)).shape

In [ ]:
types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

classes = dataset.__dict__['classes']

legend = [cls + '\n' + types_dict[cls] for cls in classes]

for n_embedded in tqdm([None, -4096, -6144,-7000, -8000, -8100]):
    for dim in [None, -1, -2, -3, -4, -5, -6, -7]:

        embs_t, embs_b = Embeddings.get_vqvae2_embs_adaptive(model, dataset, n_embedded_l=n_embedded, dim_l=dim)

        seed = 52
        np.random.seed(seed)
        torch.manual_seed(seed)

        umap_2d = UMAP(random_state=seed,n_components=2)

        umaped_vct_tb=umap_2d.fit_transform(np.concatenate([embs_t,embs_b],axis=1))

        colors = ['b', 'g', 'y', 'm', 'c']
        markers = ['8', 'v', 's', 'd', '*', ]

        embs_scatter=umaped_vct_tb
        dot_size=20
        fontsize=15
        
        r_shape=(5, 3240, 2)

        scatter_xy = embs_scatter.reshape(r_shape)

        fig, ax = plt.subplots(figsize=(10, 10))
        for i in range(len(scatter_xy)):
            ax.scatter(scatter_xy[i, :, 0], scatter_xy[i, :, 1], color=colors[i], s=dot_size,
                       marker=markers[i])


        ax.legend(legend, fontsize=fontsize)

        plt.savefig(f'embs/t_adaptive_b_static_n_embedded={n_embedded}_dim={dim}_full.png', bbox_inches = 'tight')


        plt.show()


        fig, ax = plt.subplots(figsize=(10, 10))
        for i in range(len(scatter_xy)):
            ax.scatter(scatter_xy[i, :, 0], scatter_xy[i, :, 1], color=colors[i], s=dot_size,
                       marker=markers[i])



        ax.set_xticks([])
        ax.set_yticks([])
        plt.savefig(f'embs/t_adaptive_b_static_n_embedded={n_embedded}_dim={dim}.png', bbox_inches = 'tight')
        plt.show()

In [ ]:
embs_scatter.shape

## Features spectrum

In [ ]:
embs_t=np.load('data/embs/embs_t_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy')
embs_b=np.load('data/embs/embs_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy')

In [ ]:
embs_b.shape

In [ ]:
embs_tb=np.concatenate([embs_t,embs_b],axis=1)
embs_tb.shape

In [ ]:
plt.hist(embs_b.flatten(), bins=80)
plt.show()

In [ ]:
plt.hist(embs_t.flatten(), bins=80)
plt.show()

In [ ]:
plt.hist(embs_tb.flatten(), bins=80)
plt.show()

In [ ]:
seed = 51
np.random.seed(seed)
torch.manual_seed(seed)

_,embs=train_test_split(embs_t,test_size=0.505679)

In [ ]:
embs.shape

In [ ]:
eigenvalues_o, eigenvectors_o=np.linalg.eig(embs)

In [ ]:
eigenvalues_tmp=[]
eigenvectors_tmp=[]

for i,value in tqdm(enumerate(eigenvalues_o)):
    if  value.real>-50  and value.imag <2 and value.real >-2.5 and value.real<4 and value.imag>-3:
        eigenvalues_tmp.append(value)
        eigenvectors_tmp.append(eigenvectors_o[i])

eigenvalues=np.array(eigenvalues_tmp)
eigenvectors=np.array(eigenvectors_tmp)

In [ ]:
plt.scatter(eigenvalues.real, eigenvalues.imag)
plt.show()

In [ ]:
eigenvalues_with_index=np.concatenate([eigenvalues.reshape((-1,1)), np.arange(len(eigenvalues)).reshape((-1,1))],axis=1)

In [ ]:
# points=[ [0,0], [0,-2.7], [0,2.7], [2.7,0], [-2.7,0] ]
points=[ [-2, 0], [2,0], [0,2], [0,-2], [0,0] ]
points_dict= {0:0,1:0,2:0, 3:0,4:0,5:0,6:0,7:0, 8:0}
eps=0.3
umap_2d = UMAP(random_state=seed,n_components=2)

n_points=3

np.random.shuffle(eigenvalues_with_index, )

edge_values=[]

for i, point in enumerate(points):
    for value in eigenvalues_with_index:
        dists=[]
        if abs(value[0].real-point[0])<eps and abs(value[0].imag-point[1])<eps and points_dict[i]<n_points:
            edge_values.append(value)
            points_dict[i]+=1

edge_values=np.array(edge_values)

vectors=eigenvectors[edge_values[:,1].real.astype(np.int32)]
vectors_2d_imag = umap_2d.fit_transform(vectors.imag)

colors=[]
c=['red', 'blue', 'green', 'yellow', 'cyan', 'peru', 'teal', 'greenyellow', 'purple', 'deeppink']

for i,point in enumerate(points):
    colors.extend([c[i]]*n_points)

plt.scatter(edge_values[:,0].real,edge_values[:,0].imag, c=colors)

In [ ]:
x_value=eigenvalues.real
y_value=eigenvalues.imag

heatmap, xedges, yedges = np.histogram2d(x_value, y_value, bins=100)
heatmap=heatmap.T
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

heatmap = gaussian_filter(heatmap, sigma=1)

heatmap/=np.sum(heatmap)

np.random.seed(51)
umap_2d = UMAP(random_state=seed,n_components=2)

vectors=eigenvectors[edge_values[:,1].real.astype(np.int32)]
vectors_2d_real = umap_2d.fit_transform(vectors.real)
vectors_2d_imag = umap_2d.fit_transform(vectors.imag)

extent=[-2, 2, -2 ,2]
fig,axes=plt.subplots(2,2, figsize=(10,10))
axes[0,0].scatter(x_value,y_value,alpha=0.2)
axes[0,0].scatter(edge_values[:,0].real,edge_values[:,0].imag, c=colors)
axes[0,0].set_title('Eigen values')
axes[0,0].set_xlabel('Real')
axes[0,0].set_ylabel('Imag')

fig.subplots_adjust(wspace=0.1)
# heatmap_ax=axes[0,1].imshow(heatmap, extent=extent, origin='lower', cmap=plt.cm.jet)
heatmap_ax=axes[0,1].imshow(heatmap, extent=extent, origin='lower', cmap=plt.cm.jet, interpolation='nearest', norm=mpl.colors.LogNorm())
axes[0,1].set_title('Eigen values heatmap')
axes[0,1].set_xlabel('Real')
axes[0,1].set_ylabel('Imag')

cax = fig.add_axes([0.9, 0.53, 0.02, 0.35])
cb=plt.colorbar(heatmap_ax,ax=axes[0,1], orientation='vertical',cax=cax)

axes[1,0].scatter(vectors_2d_real[:, 0], vectors_2d_real[:, 1],c=colors)
axes[1,0].set_title('Eigen vectors Real')

axes[1,1].scatter(vectors_2d_imag[:, 0], vectors_2d_imag[:, 1],c=colors)
axes[1,1].set_title('Eigen vectors Imag')


# plt.savefig('eig_5.png')
plt.show()

In [ ]:
# https://stackoverflow.com/questions/2369492/generate-a-heatmap-using-a-scatter-data-set

x=eigenvalues.real
y=eigenvalues.imag

heatmap, xedges, yedges = np.histogram2d(x, y, bins=100)
heatmap=heatmap.T
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

heatmap = gaussian_filter(heatmap, sigma=5)

plt.clf()
plt.imshow(heatmap, extent=extent, origin='lower', interpolation='sinc',cmap=plt.cm.jet,)
plt.show()